In [ ]:
import numpy as np
import pandas as pd
import math
import random
import textwrap

In [ ]:
# set seed
random.seed(22)

# alphabet
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']

# encrypted message
with open('Code.txt', 'r') as file:
    e = file.read()

e = e.rstrip('\n')

# austen matrix
M = pd.read_csv('AustenCount.txt', sep = '\t', header=None)
M = pd.DataFrame(M.values, index=alphabet, columns=alphabet)


In [ ]:
M.head()

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,
a,117,6669,7227,15101,36,2776,5505,123,10110,16,...,25852,30361,34836,2725,8924,2796,274,7976,155,16327
b,2385,208,0,35,23837,0,0,27,1207,933,...,2291,911,618,7246,9,1,0,4267,0,61
c,8527,5,1843,24,14648,0,0,14953,3366,0,...,2849,48,7661,2619,0,0,0,1289,0,326
d,4539,10,8,1234,16181,149,839,337,10164,19,...,1822,2927,9,1644,535,445,0,2619,0,80200
e,20863,269,9427,32733,13287,3902,2094,782,6315,190,...,70758,23901,12353,118,9385,2360,4406,5746,13,125631


In [ ]:
# initial candidate decryption function f
f_init = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']

In [ ]:
# generate new proposal function f'
def f_prime(f):
  """
  Args:
    f: old decryption function
  Returns:
    fnew: new decryption function (list of 26 letters)
  """
  # select 2 letters randomly
  l1, l2 = random.sample(f,2)

  # make a copy of the original f
  fnew = f.copy()

  # swap l1 and l2 in the copy of f
  for i in range(len(fnew)):
    if fnew[i] == l1:
      fnew[i] = l2
    elif fnew[i] == l2:
      fnew[i] = l1

  # output swapped copy
  return fnew

In [ ]:
# calculate acceptance probability
def acceptance_prob(f1, f2, e, M):
  """
  Args:
    f1: initial f
    f2: f prime
    e: encrypted message
    M: matrix

  Returns:
    alpha: acceptance probability
  """
  sum_f1 = 0
  sum_f2 = 0
  for i in range (len(e) - 1):
    letter1 = e[i]
    letter2 = e[i+1]

    f1_letter1 = f1[alphabet.index(letter1)]
    f1_letter2 = f1[alphabet.index(letter2)]
    f2_letter1 = f2[alphabet.index(letter1)]
    f2_letter2 = f2[alphabet.index(letter2)]

    sum_f1 += math.log(M.loc[f1_letter1, f1_letter2] + 1)
    sum_f2 += math.log(M.loc[f2_letter1, f2_letter2] + 1)

  alpha = math.exp(min(0, sum_f2 - sum_f1))
  return alpha


In [ ]:
# decode!
def decryption(n, e, M, f_init):
  f1 = f_init


  for i in range(n): # repeat n times
    f2 = f_prime(f1)
    alpha = acceptance_prob(f1, f2, e, M)
    u = random.uniform(0,1)

    if u < alpha: # accept proposal
      f1 = f2

  # decrypted message
  translation = ''.join(f1[alphabet.index(char)] for char in e if char in alphabet)
  wrapped_translation = textwrap.fill(translation, width=100)

  return ''.join(f1), wrapped_translation


In [ ]:
# test with 100 iterations
decryption(100, e, M, f_init)

('sxojewtdynkqhlamibzrgcpufv ',
 'nounsinounsinokciuiynuioru o cnlounsinogiwho choyr ciuorwgouiyltio chowrsionuofyo cnlokfe own obiobl\notknuwoshoendiorwgofeeownoenwaiuobiorozrmlei o ftobl o chowrsio cr oftoshoiwisho cnloru o chtieyo\ncnlacown orosnw raliokcr tosnw raliof oftownuocrwgownuoynn ownuorusownuoyrziownuorwhon ciuomru\nobienwafwao norosrwonobiotnsion ciuowrsiokcr tofworowrsio cr okcfzcokiozreeorountiobhorwhon\nciuowrsioknlegotsieeortotkii otnounsinoknlegokiuiociown ounsinozreegoui rfwo cr ogiruomiuyiz\nfnwokcfzcocionkitokf cnl o cr o f eiounsinognyyo chowrsiorwgoynuo cr owrsiokcfzcoftownomru onyo ciio\nrvioreeoshtiey')

In [ ]:
# test with a range of values for number of iterations
N = [100,200,500,1000,2000,5000,10000,20000] # approx 9 minutes to run
results = []

for iter in N:
  print(f"Running with n = {iter}:\n")
  final, translation = decryption(iter, e, M, f_init)
  results.append({'iterations': iter,
                  'final': final,
                  'translation': translation})

  print(f"Iterations: {iter}")
  print(f"Final Key: {final}")
  print(f"Translation: \n{translation}")
  print("-" * 100)


Running with n = 100:

Iterations: 100
Final Key: aq jsdxvglcbkeywithrumzfnpo
Translation: 
l flail flail cmifiglfi rfo omle flail uidk omk gromif rdu figexi omk drai lf ng omle cnso dlo ti
teo xclfd ak slvi rdu nss dl sldyif ti r hrwesio onx teo omk drai omro nx ak idiak omle rfo omkxisg
omleym dlo r aldoryei cmrox aldoryei no nx dlf mrdu dlf gllo dlf rfa dlf grhi dlf rdk lomif wrfo
tisldyndy ol r ard l ti xlai lomif drai cmrox nd r drai omro cmnhm ci hrss r flxi tk rdk lomif drai
clesu xaiss rx xciio xl flail clesu cifi mi dlo flail hrssu fiornd omro uirf wifgihonld cmnhm mi
lcix cnomleo omro onosi flail ulgg omk drai rdu glf omro drai cmnhm nx dl wrfo lg omii orpi rss
akxisg
----------------------------------------------------------------------------------------------------
Running with n = 200:

Iterations: 200
Final Key: ix zlnhpdsmjgyfcewbokuvarqt
Translation: 
s asies asies mueaedsae oat tusy asies keng tug dotuea onk aedyhe tug noie sa rd tusy mrlt nst we
wyt hmsan ig lspe onk 

In [ ]:
# test with a range of values for number of iterations
N = [100,1000,20000] # approx 9 minutes to run
results = []

for iter in N:
  print(f"Running with n = {iter}:\n")
  final, translation = decryption(iter, e, M, f_init)
  results.append({'iterations': iter,
                  'final': final,
                  'translation': translation})

  print(f"Iterations: {iter}")
  print(f"Final Key: {final}")
  print(f"Translation: \n{translation}")
  print("-" * 100)


Running with n = 100:

Iterations: 100
Final Key: aq jsdxvglcbkeywithrumzfnpo
Translation: 
l flail flail cmifiglfi rfo omle flail uidk omk gromif rdu figexi omk drai lf ng omle cnso dlo ti
teo xclfd ak slvi rdu nss dl sldyif ti r hrwesio onx teo omk drai omro nx ak idiak omle rfo omkxisg
omleym dlo r aldoryei cmrox aldoryei no nx dlf mrdu dlf gllo dlf rfa dlf grhi dlf rdk lomif wrfo
tisldyndy ol r ard l ti xlai lomif drai cmrox nd r drai omro cmnhm ci hrss r flxi tk rdk lomif drai
clesu xaiss rx xciio xl flail clesu cifi mi dlo flail hrssu fiornd omro uirf wifgihonld cmnhm mi
lcix cnomleo omro onosi flail ulgg omk drai rdu glf omro drai cmnhm nx dl wrfo lg omii orpi rss
akxisg
----------------------------------------------------------------------------------------------------
Running with n = 1000:

Iterations: 1000
Final Key: nx vliyjfowzdugcebmakrqshpt
Translation: 
o soneo soneo wresefose ast trou soneo keid trd fatres aik sefuye trd iane os hf trou whlt iot be
but ywosi nd loje ai